# GATK Variant Calling Pipelines

In [ ]:
# telepítések


git clone https://github.com/broadinstitute/picard.git
cd picard/
./gradlew shadowJar

cd ..
git clone https://github.com/broadinstitute/gatk.git   
cd gatk
./gradlew installAll  

cd ..
wget https://github.com/RealTimeGenomics/rtg-tools/releases/download/3.9/rtg-tools-3.9-linux-x64.zip
unzip rtg-tools-3.9-linux-x64.zip


#https://cloud.google.com/sdk/docs/
wget https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-sdk-194.0.0-linux-x86_64.tar.gz

tar xzf google-cloud-sdk-194.0.0-linux-x86_64.tar.gz

./google-cloud-sdk/install.sh
./google-cloud-sdk/bin/gcloud init

# https://hail.is/ érdemes lenne megnézni, hogy mi is ez

In [ ]:
# adatbázisok

http://cancer.sanger.ac.uk/cosmic
    
# Genome Aggregation Database (gnomAD)
http://gnomad.broadinstitute.org/downloads
# gsutil -m cp -r gs://gnomad-public/release/2.0.2/vcf/genomes/gnomad.genomes.r2.0.2.sites.chr20.vcf.bgz gnomad_data

# Panel of Normals (PoN)

# germline population frequency = G Pop Freqs


In [ ]:

export PATH=${PATH}:/home/sn/Downloads/install/src/gatk

PICARD="/home/sn/Downloads/install/src/picard/build/libs/picard.jar"

export PATH=${PATH}:/home/sn/Downloads/install/rtg-tools-3.9

# /home/sn/Downloads/install/IGV_2.4.10/igv.sh

cd /home/sn/Desktop/GATK_Vcall/wd

## Germline Variant Calling

![title](https://us.v-cdn.net/5019796/uploads/editor/mz/tzm69d8e2spl.png)


In [ ]:
# Germline Variants

java -jar $PICARD CreateSequenceDictionary R=ref.fasta O=ref.dict

samtools faidx ref.fasta

samtools index mother.bam mother.bai
samtools index father.bam father.bai
samtools index son.bam son.bai

# a htslib (samtools) telepítéssel jönnek
bgzip -c motherGIABsnps.vcf > motherGIABsnps.vcf.gz
tabix -p vcf motherGIABsnps.vcf.gz


gatk BaseRecalibrator \
  -R ref.fasta \
  -I mother.bam \
  --known-sites motherGIABsnps.vcf.gz \
  -O 1st_recal.table

  
gatk ApplyBQSR \
  -R ref.fasta \
  -I mother.bam \
  -bqsr 1st_recal.table \
  -O mother_bqsr.bam
  
    
gatk BaseRecalibrator \
  -R ref.fasta \
  -I mother_bqsr.bam \
  --known-sites motherGIABsnps.vcf.gz \
  -O 2nd_recal.table


gatk AnalyzeCovariates \
  -before 1st_recal.table \
  -after 2nd_recal.table \
  -plots plots.pdf


gatk HaplotypeCaller \
  -R ref.fasta \
  -I mother.bam \
  -O mother01.vcf \
  -L 20:10,000,000-10,200,000
  
  
# realigned reads
gatk HaplotypeCaller \
  -R ref.fasta \
  -I mother.bam \
  -O mother01debug.vcf \
  -bamout mother01debug.bam \
  -L 20:10,002,371-10,002,546 -ip 100

        
# Variant calling in gVCF mode
gatk HaplotypeCaller \
  -R ref.fasta \
  -I mother.bam \
  -O mother.g.vcf \
  -ERC GVCF \
  -L 20:10,000,000-10,200,000
  

gatk HaplotypeCaller \
  -R ref.fasta \
  -I son.bam \
  -O son.g.vcf \
  -bamout son01debug.bam \
  -ERC GVCF \
  -L 20:10,000,000-10,200,000

    
gatk HaplotypeCaller \
  -R ref.fasta \
  -I father.bam \
  -O father.g.vcf \
  -bamout father01debug.bam \
  -ERC GVCF \
  -L 20:10,000,000-10,200,000

###########################################################################  
# GVCF-based joint calling
###########################################################################
# https://software.broadinstitute.org/gatk/documentation/article.php?id=3893  

# Consolidate GVCFs using GenomicsDBImport
gatk GenomicsDBImport \
  -V mother.g.vcf \
  -V father.g.vcf \
  -V son.g.vcf \
  --genomicsdb-workspace-path csalad \
  --intervals 20:10,000,000-10,200,000

# Joint call
# Joint Genotyping
gatk GenotypeGVCFs \
  -R ref.fasta \
  -V gendb://csalad \
  -O csaladGGVCF.vcf \
  -L 20:10,000,000-10,200,000 

        
###########################################################################  
# Filter Variants
###########################################################################

# Variant Recalibration
gatk VariantRecalibrator \
  -R human.fasta \
  -V raw.SNPs.vcf \
  -resource [tags]:filename.vcf \
  -an DP -an QD -an FS -an MQRankSum {...} \
  -mode SNP \
  -recal-file raw.SNPs.recal \
  -tranches-file raw.SNPs.tranches \
  -rscript-file recal.plots.R


# ApplyVQSR applies the filtering threshold
gatk ApplyVQSR \
  -R human.fasta \
  -V raw.vcf \
  -mode SNP \
  -recal-file raw.SNPs.recal \
  -tranches-file raw.SNPs.tranches \
  -O recal.SNPs.vcf \
  -ts-filter-level 99.0


################################################ megnézni: Convolutional Neural Networks    

###########################################################################  
# Refine Genotypes
###########################################################################

# Genotype call
gatk CalculateGenotypePosteriors \
  -R reference.fasta \
  -V input.vcf \
  -ped family.ped \
  -supporting population.vcf \
  -O output.vcf

# Variants with Posterior Qualities
gatk VariantFiltration \
  -R reference.fasta \
  -V input.vcf \
  --filter-expression “GQ<20” \
  --filter-name “lowGQ” \
  -O output.vcf    
    
# Annotate Variants
gatk VariantAnnotator \
  -R reference.fasta \
  -V input.vcf \
  -A PossibleDeNovo \
  -O output.vcf


## Somatic Variant Calling

![title](https://us.v-cdn.net/5019796/uploads/editor/nk/6t0uqznajjmg.png)

https://software.broadinstitute.org/gatk/documentation/article?id=11136

https://gatkforums.broadinstitute.org/dsde/discussion/11136/how-to-call-somatic-mutations-using-gatk4-mutect2/p1?new=1

https://software.broadinstitute.org/gatk/documentation/tooldocs/current/org_broadinstitute_hellbender_tools_walkers_mutect_Mutect2.php

https://github.com/gatk-workflows/gatk4-somatic-snvs-indels

In [ ]:
cd /media/sn/oldsys/gatk_bundle/3-somatic

# T-N Pair
gatk Mutect2 \
  -R ref/Homo_sapiens_assembly38.fasta \
  -I bams/tumor.bam \
  -I bams/normal.bam \
  -tumor HCC1143_tumor \
  -normal HCC1143_normal \
  -pon resources/chr17_m2pon.vcf.gz \
  --germline-resource resources/chr17_af-only-gnomad_grch38.vcf.gz \
  --af-of-alleles-not-in-resource 0.0000025 \
  --disable-read-filter MateOnSameContigOrNoMappedMateReadFilter \
  -L resources/chr17plus.interval_list \
  -O sandbox/1_somatic_m2.vcf.gz \
  -bamout sandbox/2_tumor_normal_m2.bam

# The Panel of Normals (PoN) has a vital role that fills a gap between the matched normal and the population resource. 
# Mutect2 uses the PoN (i) to catch additional sites of noise in sequencing # data, like mapping artifacts or other 
# somewhat random but systematic artifacts of sequencing, and data processing and (ii) to filter sites of common 
# germline variation, especially for cases where a common germline variants resource is unavailable. 
# Ideally, the PoN should include samples that were sequenced on the same platform using the same chemistry and 
# analyzed using the same reference genome and workflow (technically similar). However, even an unmatched PoN is 
# better than no PoN at all. This is because mapping artifacts and polymerase slippage errors occur for pretty
# much the same genomic loci for short read sequencing approaches. 

# To make your own PoN, run Mutect2 in "tumor-only mode" on each normal BAM individually then 
# run CreateSomaticPanelOfNormals on the resulting VCFs.

#Run this command on each normal sample that will be included in the PoN
gatk Mutect2 \
  -R ref/Homo_sapiens_assembly38.fasta \
  -I bams/HG00190.bam \
  -tumor HG00190 \
  --disable-read-filter MateOnSameContigOrNoMappedMateReadFilter \
  -L resources/chr17plus.interval_list \
  -O sandbox/3_HG00190.vcf.gz


# Collate all the normal VCFs into a single callset with CreateSomaticPanelOfNormals
gatk CreateSomaticPanelOfNormals \
  -vcfs sandbox/3_HG00190.vcf.gz \
  -vcfs sandbox/4_NA19771.vcf.gz \
  -vcfs sandbox/5_HG02759.vcf.gz \
  -O sandbox/6_threesamplepon.vcf.gz


# cross-sample contamination
gatk GetPileupSummaries \
  -I bams/tumor.bam \
  -V resources/chr17_small_exac_common_3_grch38.vcf.gz \
  -O sandbox/7_tumor_getpileupsummaries.table


# Estimate contamination with CalculateContamination
gatk CalculateContamination \
  -I sandbox/7_tumor_getpileupsummaries.table \
  -O sandbox/8_tumor_calculatecontamination.table


# filtering on annotations and contamination
gatk FilterMutectCalls \
  -V sandbox/1_somatic_m2.vcf.gz \
  --contamination-table sandbox/8_tumor_calculatecontamination.table \
  -O sandbox/9_somatic_oncefiltered.vcf.gz


#CollectSequencingArtifactMetrics (Picard)
#https://software.broadinstitute.org/gatk/documentation/tooldocs/current/picard_analysis_artifacts_CollectSequencingArtifactMetrics.php
java -jar $PICARD CollectSequencingArtifactMetrics \
  I= bams/tumor.bam \
  O= sandbox/tumor.preadapter_detail_metrics \
  R= ref/Homo_sapiens_assembly38.fasta
    

# optionally filter by orientation bias
gatk FilterByOrientationBias \
  -V sandbox/9_somatic_oncefiltered.vcf.gz \
  --artifact-modes 'G/T' \
  -P sandbox/tumor.preadapter_detail_metrics \
  -O sandbox/10_tumor_oxog_twicefiltered.vcf.gz



![title](https://us.v-cdn.net/5019796/uploads/editor/x2/rzka6s5hie5r.png)

### Somatic Variant Calling with Tumor Samples Only

https://gatkforums.broadinstitute.org/gatk/discussion/6469/somatic-variant-calling-with-tumor-samples-only



# VarDict

https://www.nature.com/articles/srep43169?WT.feed_name=subjects_biotechnology
https://academic.oup.com/nar/article/44/11/e108/2468301

https://github.com/AstraZeneca-NGS/VarDict


In [ ]:
git clone https://github.com/AstraZeneca-NGS/VarDict.git

export PATH=${PATH}:/home/sn/Downloads/install/src/VarDict


# R

http://bioconductor.org/packages/release/bioc/vignettes/VariantTools/inst/doc/tutorial.pdf

http://bioconductor.org/packages/release/bioc/html/gmapR.html

https://cran.r-project.org/web/packages/vcfR/vignettes/intro_to_vcfR.html

https://www.bioconductor.org/packages/release/BiocViews.html#___SomaticMutation


# Tumor only

Evaluating somatic tumor mutation detection without matched normal samples

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5584341/